In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import tensorflow as tf

2024-09-24 19:07:45.204534: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 19:07:45.598712: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 19:07:45.599474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-24 19:08:18.145644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_dataset(directory):
    image_paths = []
    
    files = os.listdir(directory)
    
    # Here we simply filter out files that are hidden or do not end with .grb
    for filename in files:
        if not filename.startswith('.') and filename.endswith('.grb'):
            image_path = os.path.join(directory, filename)
            image_paths.append(image_path)
        
    return image_paths

train_directory = "/pool/data/ERA5/E5/sf/an/1D/167/"
train = pd.DataFrame()
train['data'] = load_dataset(train_directory)

In [3]:
len(train)

1014

In [4]:
# shuffle the dataset
control = 'data'
random_order = np.random.permutation(len(train))
train['RandomOrder'] = random_order
train = train.sort_values(by=['RandomOrder', control]).reset_index(drop=True)
train = train.drop(columns=['RandomOrder'])

In [5]:
train_feature_paths = train['data'].values

In [6]:
train_feature_paths

array(['/pool/data/ERA5/E5/sf/an/1D/167/E5sf00_1D_1992-11_167.grb',
       '/pool/data/ERA5/E5/sf/an/1D/167/E5sf00_1D_1942-01_167.grb',
       '/pool/data/ERA5/E5/sf/an/1D/167/E5sf00_1D_1983-05_167.grb', ...,
       '/pool/data/ERA5/E5/sf/an/1D/167/E5sf00_1D_1991-03_167.grb',
       '/pool/data/ERA5/E5/sf/an/1D/167/E5sf00_1D_1981-07_167.grb',
       '/pool/data/ERA5/E5/sf/an/1D/167/E5sf00_1D_1967-07_167.grb'],
      dtype=object)

In [7]:
def generate_data(train_feature_paths, idx,  start_time, end_time):
    for  file_path in train_feature_paths:
        try:
            dataset = xr.open_dataset(file_path, engine='cfgrib', backend_kwargs={'indexpath': ''})
            dataset = dataset.sel(time=slice(np.datetime64(start_time), np.datetime64(end_time)))
            feature = dataset['t2m'][idx].values.reshape(-1, 640)
            feature = np.expand_dims(feature, axis=-1)
            label = int(np.datetime_as_string(dataset['valid_time'][idx].values)[5:7]) - 1
            feature = tf.image.resize(feature, [224, 224]).numpy() # data augmentation if needed, not compulsory
            yield feature, label

        except Exception as e:
            continue

def create_dataset(train_feature_paths, idx, start_time='1960-01-01', end_time='2000-01-01', batch_size=32):
    dataset = tf.data.Dataset.from_generator(lambda: generate_data(train_feature_paths, idx, start_time, end_time),
                                             output_signature=(tf.TensorSpec(shape=(224, 224, 1), dtype=tf.float32),
                                                               tf.TensorSpec(shape=(), dtype=tf.int32)))
    dataset = dataset.batch(batch_size)#.repeat()
    return dataset#.shuffle(buffer_size=32).repeat()

In [8]:
# Create simple model
model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
                             tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(12, activation='softmax')])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [9]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 128)               50466944  
                                                                 
 dense_1 (Dense)             (None, 12)                1548      
                                                                 
Total params: 50468812 (192.52 MB)
Trainable params: 50468812 (192.52 MB)
Non-trainable params: 0 (0.00 Byte)
____________

In [10]:
for idx in range(7): # each image file has multiple files hence the continuos training based on indices
    dataset = create_dataset(train_feature_paths, idx)
    model.fit(dataset, epochs=10)

Epoch 1/10
15/15 [==============================] - 94s 6s/step - loss: 6790.5688 - accuracy: 0.0958
Epoch 2/10
15/15 [==============================] - 89s 6s/step - loss: 1673.6545 - accuracy: 0.0729
Epoch 3/10
15/15 [==============================] - 92s 6s/step - loss: 275.1270 - accuracy: 0.0708
Epoch 4/10
15/15 [==============================] - 92s 6s/step - loss: 2.7272 - accuracy: 0.0854
Epoch 5/10
15/15 [==============================] - 108s 7s/step - loss: 2.4850 - accuracy: 0.0833
Epoch 6/10
15/15 [==============================] - 91s 6s/step - loss: 18.4287 - accuracy: 0.0875
Epoch 7/10
15/15 [==============================] - 92s 6s/step - loss: 2.4850 - accuracy: 0.0833
Epoch 8/10
15/15 [==============================] - 95s 6s/step - loss: 2.4850 - accuracy: 0.0833
Epoch 9/10
15/15 [==============================] - 96s 6s/step - loss: 2.4850 - accuracy: 0.0833
Epoch 10/10
15/15 [==============================] - 94s 6s/step - loss: 2.4850 - accuracy: 0.0833
Epoch 1/1